In [1]:
# Import Splinter, BeautifulSoup, and Pandas
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Import Splinter, BeautifulSoup, and Pandas
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
# Set the executable path and initialize Splinter
executable_path = {'executable_path': ChromeDriverManager(version = '98.0.4758.102').install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Trying to download new driver from https://chromedriver.storage.googleapis.com/98.0.4758.102/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\M037228\.wdm\drivers\chromedriver\win32\98.0.4758.102]


### Visit the NASA Mars News Site

In [3]:
# Visit the mars nasa news site
url = 'https://redplanetscience.com/'
browser.visit(url)

# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [4]:
# Convert the browser html to a soup object and then quit the browser
html = browser.html
news_soup = bs(html, 'html.parser')

slide_elem = news_soup.select_one('div.list_text')

In [5]:
slide_elem.find('div', class_='content_title')

<div class="content_title">Sensors on Mars 2020 Spacecraft Answer Long-Distance Call From Earth</div>

In [6]:
# Use the parent element to find the first a tag and save it as `news_title`
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

'Sensors on Mars 2020 Spacecraft Answer Long-Distance Call From Earth'

In [7]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

"Instruments tailored to collect data during the descent of NASA's next rover through the Red Planet's atmosphere have been checked in flight."

### JPL Space Images Featured Image

In [8]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [9]:
# Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [10]:
# Parse the resulting html with soup
html = browser.html
img_soup = bs(html, 'html.parser')
img_soup

<html class="fancybox-margin fancybox-lock"><head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" rel="stylesheet"/>
<!-- <link rel="stylesheet" type="text/css" href="css/font.css"> -->
<link href="css/app.css" rel="stylesheet" type="text/css"/>
<link href="https://stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css" rel="stylesheet" type="text/css"/>
<title>Space Image</title>
<style type="text/css">.fancybox-margin{margin-right:17px;}</style></head>
<body>
<div class="header">
<nav class="navbar navbar-expand-lg">
<a class="navbar-brand" href="#"><img id="logo" src="image/nasa.png"/><span class="logo">Jet Propulsion Laboratory</span>
<span class="logo1">California Institute of Technology</span></a>
<button aria-controls="navbarNav" aria-expanded="false" aria-label="Toggle navigation" class="navbar-toggler" data-target="#navbarNav" data-t

In [11]:
# find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars3.jpg'

In [12]:
# Use the base url to create an absolute url
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars3.jpg'

### Mars Facts

In [13]:
df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.head()

,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"


In [14]:
df.columns=['Description', 'Mars', 'Earth']
df.set_index('Description', inplace=True)
df

,Mars,Earth
Description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [15]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

# D1: Scrape High-Resolution Mars’ Hemisphere Images and Titles

### Hemispheres

In [1]:
# Import Splinter, BeautifulSoup, and Pandas
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
# Set the executable path and initialize Splinter
executable_path = {'executable_path': ChromeDriverManager(version = '98.0.4758.102').install()}
browser = Browser('chrome', **executable_path, headless=False)
# 1. Use browser to visit the URL 
url = 'https://marshemispheres.com/'

browser.visit(url)



====== WebDriver manager ======
Trying to download new driver from https://chromedriver.storage.googleapis.com/98.0.4758.102/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\melet\.wdm\drivers\chromedriver\win32\98.0.4758.102]


In [20]:
## This gets the list of titles

# # 2. Create a list to hold the images and titles.
hemisphere_image_urls = []

url_root = 'https://marshemispheres.com/'
# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')
# Retrieve all elements that contain book information
#articles = soup.find_all('article', class_='product_pod')
articles = soup.find_all('div', class_='item')

url_list = []
title_list = []

title_list = soup.find_all('h3') #get href as list of links, use browser.visit instead of click

for i in title_list[:4]:
    hemispheres = {}
    title = i.text.strip()
    hemispheres['title'] = title
    browser.links.find_by_partial_text(title).click()
    link = browser.links.find_by_text('Sample').first ## THIS ONE WORKS
    img_url = link['href']
    hemispheres['link'] = img_url
    hemisphere_image_urls.append(hemispheres)

    browser.back()

print(hemisphere_image_urls)



StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=98.0.4758.102)
Stacktrace:
Backtrace:
	Ordinal0 [0x00CB69A3+2582947]
	Ordinal0 [0x00C4A6D1+2139857]
	Ordinal0 [0x00B43A98+1063576]
	Ordinal0 [0x00B462B7+1073847]
	Ordinal0 [0x00B4617E+1073534]
	Ordinal0 [0x00B46A2A+1075754]
	Ordinal0 [0x00B98C89+1412233]
	Ordinal0 [0x00B8854C+1344844]
	Ordinal0 [0x00B9834A+1409866]
	Ordinal0 [0x00B88366+1344358]
	Ordinal0 [0x00B65176+1200502]
	Ordinal0 [0x00B66066+1204326]
	GetHandleVerifier [0x00E5BE02+1675858]
	GetHandleVerifier [0x00F1036C+2414524]
	GetHandleVerifier [0x00D4BB01+560977]
	GetHandleVerifier [0x00D4A8D3+556323]
	Ordinal0 [0x00C5020E+2163214]
	Ordinal0 [0x00C55078+2183288]
	Ordinal0 [0x00C551C0+2183616]
	Ordinal0 [0x00C5EE1C+2223644]
	BaseThreadInitThunk [0x75EEFA29+25]
	RtlGetAppContainerNamedObjectPath [0x774C7A9E+286]
	RtlGetAppContainerNamedObjectPath [0x774C7A6E+238]


In [22]:
    lev1 = browser.find_by_css('div')
    
    #link = browser.find('a') #ERROR: webdriver object has no attribute "find"
    #link = browser.find_by_tag('a') # this give a list of splinter element lists
    #link = browser.find_by_tag('a').first # this give a list of splinter driver webdriver object
    #lev1 = browser.links.find_by_partial_href('images/full')# WHY DOESNT THIS WORK: stale element reference: element is not attached to the page document
    #lev1 = browser.find_by_tag('li') # ERROR: stale element reference: element is not attached to the page document
    lev1 = browser.find_by_css('div')
    # browser.find
    # lev1 = browser.find_by_value('images') #stale element reference: element is not attached to the page document
    print(lev1)
    # lev1 = browser.find_by_value('images') #stale element reference: element is not attached to the page document
    print(lev1)

TypeError: find_by_css() got an unexpected keyword argument 'class_'

In [63]:
## This gets the list of titles

# # 2. Create a list to hold the images and titles.
hemisphere_image_urls = []

url_root = 'https://marshemispheres.com/'
# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')
# Retrieve all elements that contain book information
#articles = soup.find_all('article', class_='product_pod')
articles = soup.find_all('div', class_='item')

url_list = []
title_list = []
testlist = []

    # Iterate through each book
title_list = soup.find_all('h3')
for i in title_list[:4]:
    hemispheres = {}
    title = i.text.strip()
    hemispheres['title'] = title
    

    browser.links.find_by_partial_text(title).click()
    #################################################
    #?????????????????????????????????
    link = browser.links.find_by_partial_text("src") #this just gives me a bunch of splinter element lists??


    # link = browser.links.find_by_text("full").first
    # print(browser.links.find_by_partial_text("full"))
    # img_url = link['href']

    # hemispheres['link'] = img_url
    # hemisphere_image_urls.append(hemispheres)

    browser.back()

link.first

ElementDoesNotExist: no elements could be found with link by partial text "src"

In [26]:
####################################################
# ??????????????????????????


for title in title_list:    
    browser.links.find_by_partial_text(title.text.strip()).click()
    for link in soup.find_all('li'):
        # link2 = link.a('href')
        item = link.find('img', class_='thumb')
        print(item)
    browser.back()


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=99.0.4844.51)
Stacktrace:
Backtrace:
	Ordinal0 [0x005669A3+2582947]
	Ordinal0 [0x004FA6D1+2139857]
	Ordinal0 [0x003F3960+1063264]
	Ordinal0 [0x00420E4E+1248846]
	Ordinal0 [0x00416B53+1207123]
	Ordinal0 [0x0043854C+1344844]
	Ordinal0 [0x00416524+1205540]
	Ordinal0 [0x004386A4+1345188]
	Ordinal0 [0x0044834A+1409866]
	Ordinal0 [0x00438366+1344358]
	Ordinal0 [0x00415176+1200502]
	Ordinal0 [0x00416066+1204326]
	GetHandleVerifier [0x0070BE02+1675858]
	GetHandleVerifier [0x007C036C+2414524]
	GetHandleVerifier [0x005FBB01+560977]
	GetHandleVerifier [0x005FA8D3+556323]
	Ordinal0 [0x0050020E+2163214]
	Ordinal0 [0x00505078+2183288]
	Ordinal0 [0x005051C0+2183616]
	Ordinal0 [0x0050EE1C+2223644]
	BaseThreadInitThunk [0x7682FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77557A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77557A6E+238]


In [50]:
####################################################
# ??????????????????????????
# book_url = category.find('a')['href']

# title = soup.find_all('img', class_="thumb")
# for i in title:
#     print(i.text.strip())
#     print('(------)')
#     print(title)

####################################################
# ??????????????????????????
# y = soup.find_all('dd')
# y = soup.find_all('div', class_="description")
# x = y.find_all('a')
# link = y.find('a')['href']
# print(link)
element = browser.links.find_by_text('Sample').first
print(element)

y = soup.find_all('a', target="_blank")
print(y)

ElementDoesNotExist: no elements could be found with link by text "Sample"

In [27]:
####################################################
# ??????????????????????????

for link in soup.find_all('a'):
    link.get('href')
    print(link.get('href'))

#
#
https://astrogeology.usgs.gov/search
cerberus.html
cerberus.html
schiaparelli.html
schiaparelli.html
syrtis.html
syrtis.html
valles.html
valles.html
#
#
#
#
#


In [40]:
# #     browser.links.find_by_partial_text(title).click()
    for link in soup.find_all('a'):
        print(link.get('href'))

    # browser.back()

# print(planets)


# image_urls = ['https://marshemispheres.com/' + url for url in url_list]

https://marshemispheres.com/images/full.jpg


In [7]:
print(url_list)

[]


In [1]:
# # 2. Create a list to hold the images and titles.
hemisphere_image_urls = []

# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')
# Retrieve all elements that contain book information
#articles = soup.find_all('article', class_='product_pod')
articles = soup.find_all('div', class_='item')
# 2. Create a list to hold the images and titles.
hemisphere_image_urls = []

# 3. Write code to retrieve the image urls and titles for each hemisphere.
for i in range(4):
    #create empty dictionary
    hemispheres = {}
    browser.find_by_css('a.product-item h3')[i].click()
    element = browser.links.find_by_text('Sample').first
    print(element)
    img_url = element['href']
    print(img_url)
    title = browser.find_by_css("h2.title").text
    print(title)
#    hemispheres["img_url"] = img_url
#    hemispheres["title"] = title
#    hemisphere_image_urls.append(hemispheres)
    browser.back()

#print(hemisphere_image_urls)

    

NameError: name 'browser' is not defined

In [ ]:
# Import Splinter, BeautifulSoup, and Pandas
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
# Set the executable path and initialize Splinter
executable_path = {'executable_path': ChromeDriverManager(version = '98.0.4758.102').install()}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(url)
# # 2. Create a list to hold the images and titles.
hemisphere_image_urls = []
hemispheres = {}
url_root = 'https://marshemispheres.com/'
# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')
# Retrieve all elements that contain book information
#articles = soup.find_all('article', class_='product_pod')
articles = soup.find_all('div', class_='item')
url_list = []
title_list = []
    # Iterate through each book
for article in len(articles):
# Use Beautiful Soup's find() method to navigate and retrieve attributes

    h3 = article.find('h3')
    title = h3.text.strip()
    hemispheres['title'] = title
    print(title)
    browser.links.find_by_partial_text(title).click()
    for link in soup.find_all('a'):
        print(link.get('href'))

    browser.back()


print(hemispheres)

image_urls = ['https://marshemispheres.com/' + url for url in url_list]

TypeError: 'int' object is not iterable

In [6]:
# # # 2. Create a list to hold the images and titles.
# hemisphere_image_urls = []
# hemispheres = {}
# url_root = 'https://marshemispheres.com/'
# # HTML object
# html = browser.html
# # Parse HTML with Beautiful Soup
# soup = bs(html, 'html.parser')
# # Retrieve all elements that contain book information
# #articles = soup.find_all('article', class_='product_pod')
# articles = soup.find_all('div', class_='item')
# url_list = []
# title_list = []
#     # Iterate through each book
# for article in articles:
#     h3 = article.find('h3')
#     title = h3.text.strip()







# image_urls = ['https://marshemispheres.com/' + url for url in url_list]

# print(image_urls)

[]


In [15]:
browser.visit(url_root)
# # 2. Create a list to hold the images and titles.
hemisphere_image_urls = []
hemispheres = {}
url_root = 'https://marshemispheres.com/'

html = browser.html
# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')
# Retrieve all elements that contain book information

articles = soup.find_all('div', class_='item')
print(articles)
url_list = []
title_list = []

for article in articles:
    #h3 = article.find('h3')
    title = h3.text.strip()
    print(title)
    title_list.append(title)
    browser.links.find_by_partial_text(title).click()
    x = browser.find_by_tag('h2')
    page_title = x.text.strip()
    print(page_title)
    links = []
    for link in soup.find_all('a'):
        #links.append(soup.get('href'))
        s = soup.find_all('src')
        #print(s)
        #print(link.find_all['j'])


    



        # links.append(target)
        # links.append(link.get('href'))
        # print(links)
    browser.back()
#url_list = links[3]
# print(links)


[<div class="item">
<a class="itemLink product-item" href="cerberus.html"><img alt="Cerberus Hemisphere Enhanced thumbnail" class="thumb" src="images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png"/></a>
<div class="description">
<a class="itemLink product-item" href="cerberus.html">
<h3>Cerberus Hemisphere Enhanced</h3>
</a>
<span class="subtitle" style="float:left">image/tiff 21 MB</span><span class="pubDate" style="float:right"></span><br/>
<p>Mosaic of the Cerberus hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. This mosaic is composed of 104 Viking Orbiter images acquired…</p>
</div>
<!-- end description -->
</div>, <div class="item">
<a class="itemLink product-item" href="schiaparelli.html"><img alt="Schiaparelli Hemisphere Enhanced thumbnail" class="thumb" src="images/08eac6e22c07fb1fe72223a79252de20_schiaparelli_enhanced.tif_thumb.png"/></a>
<div class="description">
<a class="itemLink product-

In [16]:
for article in articles:
    h3 = article.find('h3')
    title = h3.text.strip()
    # print(title)
    title_list.append(title)
    browser.links.find_by_partial_text(title).click()
    x = browser.find_by_tag('h2')
    page_title = x.text.strip()
    print(page_title)
    links = []
    for link in soup.find_all('li'):
        print(link)
        # if link.has_attr('target'):
        #     print(link)
    browser.back()

KeyboardInterrupt: 

In [18]:
# 4. Print the list that holds the dictionary of each image url and title.
hemisphere_image_urls

[{'img_url': 'https://marshemispheres.com/images/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [15]:
# 5. Quit the browser
browser.quit()